In [ ]:
import warnings
warnings.filterwarnings('ignore') # hide warnings
import os
import numpy as np
import sys
sys.path.append("..")
from dyntapy.supply_data import get_toy_network, relabel_graph
from dyntapy.demand_data import add_centroids, od_graph_from_matrix
from dyntapy.visualization import show_network
from dyntapy.assignments import StaticAssignment

In [ ]:
# Get network
network = get_toy_network('two-route')
centroid_x = np.array([-1, 3])
centroid_y = np.array([2, 2])
network = add_centroids(network, centroid_x, centroid_y, euclidean=True)
network = relabel_graph(network) 
# show_network(network, euclidean=True, notebook= True)


In [ ]:
# Initial OD to start iterating
old_od = np.zeros(4).reshape((2, 2))
old_od[0, 1] = 400

old_graph = od_graph_from_matrix(old_od, centroid_x, centroid_y)
tolls = np.zeros(network.number_of_edges())

assignment = StaticAssignment(network,old_graph, tolls)
result = assignment.run('dial_b')   

In [ ]:
# A matrix
A = 0.05971545
# B matrix
B = 10

In [ ]:
# First iteration
assignment = StaticAssignment(network,old_graph, tolls)
result = assignment.run('dial_b')    
skims = result.skim
new_od = np.zeros(4).reshape((2, 2))
new_od[0,1] = (A-skims)/B
print(new_od[0,1])
print(abs((new_od - old_od)))

i = 1 
max_iterations = 10
max_deviation_per_od = 5
while np.max(abs((new_od - old_od))) > max_deviation_per_od and i < max_iterations:
    # Perhaps MSA stap
    old_od = new_od
    old_graph =  od_graph_from_matrix(old_od, centroid_x, centroid_y)
    assignment = StaticAssignment(network,old_graph, tolls)
    result = assignment.run('dial_b')
    skims = result.skim
    new_od = np.zeros(4).reshape((2, 2))
    new_od[0,1] = (A-skims)/B
    i += 1
    print(abs((new_od - old_od)))
    